# Extraction

In this notebook, we will extract the breadcrumb, categories and URLs for
all products and store them in a relational data base.

In [2]:
import gzip
import re
import os
import shutil
import sqlite3
from urllib import request

In [3]:
# setup tables
conn = sqlite3.connect('products.db') 
with conn:
    conn.execute("""
        CREATE TABLE IF NOT EXISTS
            taxonomies(NODE TEXT PRIMARY KEY,
                       URL TEXT,
                       CATEGORY TEXT,
                       BREADCRUMB TEXT)
    """)
    conn.execute("""
        CREATE TABLE IF NOT EXISTS
            product_ids(NODE TEXT PRIMARY KEY,
                        SKU TEXT,
                        PRODUCT_ID TEXT,
                        MPN TEXT,
                        GTIN13 TEXT,
                        IDENTIFIER TEXT)
    """)

In [4]:
taxo_pattern = re.compile("<http://schema.org/Product/category>|<http://schema.org/Product/breadcrumb>|<http://schema.org/Product/url>", re.IGNORECASE)
id_pattern = re.compile("<http://schema.org/Product/sku>|<http://schema.org/Product/productID>|<http://schema.org/Product/mpn>|<http://schema.org/Product/gtin13>|<http://schema.org/Product/identifier>", re.IGNORECASE)
base_url = "http://data.dws.informatik.uni-mannheim.de/2018-12_schemaChunks/Product_MD/split_product."

for i in range(1, 97):
    filename = "../Data/products_" + str(i)            
    if not os.path.exists(filename + ".gz"):
        request.urlretrieve(base_url + str(i) + ".gz", filename + ".gz")
        print("successfully downloaded " + filename + ".gz")
    if not os.path.exists(filename):
        with open(filename, 'wb') as target:
            with gzip.open(filename + ".gz", 'rb') as source:
                shutil.copyfileobj(source, target)
        print("successfully extracted " + filename + ".gz")
        
    print("processing " + filename + "...")
    with open(filename, mode='r') as f:
        j = 0
        for line in iter(f.readline, ''): 
            j += 1
            if j % 10000000 == 0:
                print("processed " + str(j / 1000000) + " million lines so far")
                
            if not taxo_pattern.search(line) or not id_pattern.search(line):
                continue  # abort execution for no matches
            
            props = line.split()
            with conn:
                if taxo_pattern.search(line):
                    conn.execute("INSERT OR IGNORE INTO taxonomies (NODE) VALUES (?)", (props[0],))
                if id_pattern.search(line):
                    conn.execute("INSERT OR IGNORE INTO product_ids (NODE) VALUES (?)", (props[0],))
                
                # handle taxonomy related info
                if props[1] == "<http://schema.org/Product/category>":
                    conn.execute("UPDATE taxonomies SET CATEGORY = ? WHERE NODE = ?", (props[2], props[0]))
                if props[1] == "<http://schema.org/Product/url>":
                    conn.execute("UPDATE taxonomies SET URL = ? WHERE NODE = ?", (props[2], props[0]))
                if props[1] == "<http://schema.org/Product/breadcrumb>":
                    conn.execute("UPDATE taxonomies SET BREADCRUMB = ? WHERE NODE = ?", (props[2], props[0]))
                    
                # handle id related info
                if props[1] == "<http://schema.org/Product/sku>":
                    conn.execute("UPDATE product_ids SET SKU = ? WHERE NODE = ?", (props[2], props[0]))
                if props[1] == "<http://schema.org/Product/productID>":
                    conn.execute("UPDATE product_ids SET PRODUCT_ID = ? WHERE NODE = ?", (props[2], props[0]))
                if props[1] == "<http://schema.org/Product/mpn>":
                    conn.execute("UPDATE product_ids SET MPN = ? WHERE NODE = ?", (props[2], props[0]))
                if props[1] == "<http://schema.org/Product/gtin13>":
                    conn.execute("UPDATE product_ids SET GTIN13 = ? WHERE NODE = ?", (props[2], props[0]))
                if props[1] == "<http://schema.org/Product/identifier>":
                    conn.execute("UPDATE product_ids SET IDENTIFIER = ? WHERE NODE = ?", (props[2], props[0]))       
    
    print("remove files after extraction...")
    os.remove(filename + ".gz")
    os.remove(filename)
    print("successfully removed " + filename + "(.gz)")
            